In [1]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

# Data Loader

In [2]:
class DataLoader:
    req_columns = ['CH_TIMESTAMP', 'CH_SYMBOL', 'CH_TRADE_HIGH_PRICE', 'CH_TRADE_LOW_PRICE', 'CH_OPENING_PRICE', 'CH_CLOSING_PRICE', 'CH_LAST_TRADED_PRICE', 'CH_PREVIOUS_CLS_PRICE', 'CH_TOT_TRADED_QTY', 'CH_52WEEK_HIGH_PRICE', 'CH_52WEEK_LOW_PRICE']
    new_column_names = ['date', 'symbol', 'high', 'low', 'open', 'close', 'ltp', 'prev_close', 'volume', 'high_52w', 'low_52w']

    @staticmethod
    def load_data(script_name, start_date, end_date, series="EQ"):
        df = equity_history(script_name, series, start_date, end_date)[DataLoader.req_columns]
        df.columns = DataLoader.new_column_names
        return df

In [3]:
script_name = 'INFY'
series = 'EQ'
end_date = datetime.now().date()
start_date = end_date - timedelta(days = 370)

In [4]:
df = DataLoader.load_data(
    script_name, 
    start_date.strftime("%d-%m-%Y"), 
    end_date.strftime("%d-%m-%Y"), 
    series).sort_values('date').drop_duplicates()

In [5]:
df

,date,symbol,high,low,open,close,ltp,prev_close,volume,high_52w,low_52w
230,2021-09-06,INFY,1732.00,1699.35,1705.15,1730.40,1731.20,1700.65,4876026,1757.0,913.05
231,2021-09-07,INFY,1734.90,1704.00,1730.00,1706.65,1708.00,1730.40,3520341,1757.0,913.05
232,2021-09-08,INFY,1700.85,1682.95,1700.85,1693.25,1696.45,1706.65,4584896,1757.0,913.05
233,2021-09-09,INFY,1694.90,1682.15,1685.90,1691.60,1687.95,1693.25,3331674,1757.0,913.05
234,2021-09-13,INFY,1701.50,1675.20,1683.00,1691.90,1690.85,1691.60,4662374,1757.0,948.30
...,...,...,...,...,...,...,...,...,...,...,...
2,2022-09-05,INFY,1467.00,1446.35,1451.00,1461.30,1461.25,1453.00,3669352,1953.9,1367.15
3,2022-09-06,INFY,1470.00,1445.45,1461.30,1455.10,1456.65,1461.30,3428442,1953.9,1367.15
4,2022-09-07,INFY,1465.75,1438.05,1440.00,1457.65,1462.00,1455.10,3600574,1953.9,1367.15
5,2022-09-08,INFY,1481.10,1463.60,1475.00,1475.90,1475.05,1457.65,4061475,1953.9,1367.15


In [6]:
df['candle_body_length'] = (df['open'] - df['close']).abs()
df['candle_length'] = df['high'] - df['low']
df['candle_body_ratio'] = df['candle_body_length'] / df['candle_length']
df['candle_color'] = np.where(df.close > df.open, 'green', 'red')
df['pct_change'] = (df['close'] - df['prev_close']) * 100 / df['prev_close']

# Multiple candlestick pattern functions

In [44]:
def identify_engulfing(df):
    for i in ['open', 'high', 'close', 'low', 'candle_color', 'candle_body_ratio', 'candle_body_length']:
        df[f'prev_{i}'] = df[i].shift(1)

    df['engulfing'] = np.where(
        np.logical_and.reduce([
            df['prev_candle_color'] != df['candle_color'],
            df['prev_candle_body_ratio'] >= 0.15,
            df['prev_open'].between(df[['open', 'close']].min(axis=1), df[['open', 'close']].max(axis=1)),
            df['prev_close'].between(df[['open', 'close']].min(axis=1), df[['open', 'close']].max(axis=1)),
            (df['prev_candle_body_length'] / df['candle_body_length']) < 0.95
        ]),
        np.where(df["candle_color"] == 'green', "Bullish Engulfing", "Bearish Engulfing"),
        None
    )
    
    return df

def identify_haramis(df):
    df['harami'] = np.where(
        np.logical_and.reduce([
            df['prev_candle_color'] != df['candle_color'],
            df['prev_candle_body_ratio'] >= 0.5,
            (df['candle_body_length'] / df['prev_candle_body_length']) < 0.5,
            df['open'].between(df[['prev_open', 'prev_close']].min(axis=1), df[['prev_open', 'prev_close']].max(axis=1)),
            df['close'].between(df[['prev_open', 'prev_close']].min(axis=1), df[['prev_open', 'prev_close']].max(axis=1)),
        ]),
        np.where(df['candle_color'] == 'green', "Bullish Harami", "Bearish Harami"),
        None
    )
    return df

def identify_piercing_or_dark_clouds(df):
    df["partial_engulfing"] = np.where(
        np.logical_and.reduce([
            df["close"].between(df[["prev_open", "prev_close"]].min(axis=1), df[["prev_open", "prev_close"]].max(axis=1)),
            df["prev_candle_body_ratio"] >= 0.5,
            df['candle_body_length'] >= df['prev_candle_body_length'] * 0.5,
            df["candle_color"] != df["prev_candle_color"]
        ]),
        np.where(
            (df["candle_color"] == "green") & (df['close'] >= (df['prev_open'] + df['prev_close']) / 2), 
            "Piercing Pattern",
            np.where(
                (df["candle_color"] == "red") & (df['close'] <= (df['prev_open'] + df['prev_close']) / 2),
                "Dark Cloud Cover",
                None
            )
        ),
        None
    )
    return df

In [45]:
for fn in [identify_engulfing, identify_haramis, identify_piercing_or_dark_clouds]:
    df = fn(df)

In [46]:
df[df["partial_engulfing"].notnull()]

,date,symbol,high,low,open,close,ltp,prev_close,volume,high_52w,...,pct_change,prev_open,prev_high,prev_low,prev_candle_color,prev_candle_body_ratio,prev_candle_body_length,engulfing,harami,partial_engulfing
231,2021-09-07,INFY,1734.90,1704.00,1730.00,1706.65,1708.00,1730.40,3520341,1757.0,...,-1.372515,1705.15,1732.00,1699.35,green,0.773354,25.25,None,None,Dark Cloud Cover
237,2021-09-16,INFY,1719.25,1691.45,1715.20,1702.25,1702.05,1711.45,3246568,1757.0,...,-0.537556,1693.25,1715.10,1690.20,green,0.730924,18.20,None,None,Dark Cloud Cover
247,2021-09-30,INFY,1703.00,1670.10,1703.00,1675.20,1679.50,1692.25,6914031,1788.0,...,-1.007534,1659.25,1701.40,1655.00,green,0.711207,33.00,None,None,Dark Cloud Cover
251,2021-10-06,INFY,1709.00,1670.00,1702.10,1673.55,1672.65,1692.80,4271698,1788.0,...,-1.137169,1671.00,1702.95,1663.00,green,0.545682,21.80,None,None,Dark Cloud Cover
229,2021-11-25,INFY,1726.20,1696.95,1700.00,1722.40,1722.50,1696.00,4476260,1848.0,...,1.556604,1740.00,1740.50,1688.00,red,0.838095,44.00,None,None,Piercing Pattern
138,2022-03-15,INFY,1888.35,1830.00,1888.35,1839.30,1838.00,1890.70,7140443,1953.9,...,-2.718570,1831.55,1895.00,1831.55,green,0.932230,59.15,None,None,Dark Cloud Cover
144,2022-03-24,INFY,1894.60,1856.15,1856.15,1886.70,1881.85,1872.40,3784303,1953.9,...,0.763726,1897.00,1900.00,1857.00,red,0.572093,24.60,None,None,Piercing Pattern
84,2022-06-02,INFY,1513.85,1475.60,1484.00,1508.00,1508.00,1478.55,7205840,1953.9,...,1.991816,1513.00,1514.75,1472.10,red,0.807737,34.45,None,None,Piercing Pattern
89,2022-06-09,INFY,1516.95,1481.50,1485.00,1515.00,1514.50,1500.00,4361061,1953.9,...,1.000000,1520.20,1520.20,1492.00,red,0.716312,20.20,None,None,Piercing Pattern
34,2022-06-14,INFY,1448.90,1403.15,1410.00,1440.55,1440.95,1424.50,6090247,1953.9,...,1.126711,1443.60,1443.60,1411.70,red,0.598746,19.10,None,None,Piercing Pattern
